# Data Cleaning.

In [1]:
import pandas as pd

from src.cleaning_functions import *

from src.text_functions import *

**ÍNDICE:**
1. Nike data.
    - Exploración inicial de los datos.
    - Hashtags.
    - Preparing Data.
    - Subsets.



2. Adidas data.
    - Exploración inicial de los datos.
    - Hashtags.
    - Preparing Data.
    - Subsets.



3. Dataset Completo.


## Nike data.

In [2]:
nike = pd.read_csv('data/nike.csv')

In [3]:
nike_tw = pd.DataFrame(nike.text)

### Exploración inicial de los datos:

In [4]:
nike_tw.shape

(29804, 1)

In [5]:
nike_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29804 entries, 0 to 29803
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29804 non-null  object
dtypes: object(1)
memory usage: 233.0+ KB


In [6]:
nike_tw.describe()

,text
count,29804
unique,29276
top,@whatudohazz nike
freq,59


In [7]:
nike_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [8]:
nike_tw.duplicated().sum() # Encontramos 528 valores duplicados.

528

In [9]:
drop_duplicates(nike_tw) # Eliminamos los valores duplicados.

In [10]:
nike_tw.shape

(29276, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

*Posibles aplicaciones de este análisis:*
- *Análisis del sentimiento de la conversación en torno a un hashtag específico (nos permite analizar la percepción acerca de una campaña online, un evento en directo o similares).*


- *Ranking de hashtags con mayor sentimiento positivo o negativo.*


- *Análisis de utilización de hashtags: podemos analizar elementos más básicos como los hashtags más frecuentes o incluso cruzarlo con otros datos extraídos (datetime de publicación, usuarios que publican el tweet...) para llegar a conclusiones relevantes.*

In [11]:
nike_tw['hashtags'] = nike_tw.text.apply(extract_hashtags)

In [12]:
nike_tw.head()

,text,hashtags
0,@ZubyMusic @LPMisesCaucus Nobody cares about N...,[]
1,@PoojaPraharaj @IndiainUkraine @PMOIndia @MEAI...,[]
2,@DocumentWomen Nike Okundaye. Also knowns Nike...,"[#womengiant, #Documentwomen]"
3,Day 4 of #maxmadness #AirMaxMonth \nNike Air M...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,@MagMr44 @soleguru @nikestore @Nike @SneakerAd...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [13]:
nike_tw.head()

,text,hashtags
0,@ZubyMusic @LPMisesCaucus Nobody cares about N...,[]
1,@PoojaPraharaj @IndiainUkraine @PMOIndia @MEAI...,[]
2,@DocumentWomen Nike Okundaye. Also knowns Nike...,"[#womengiant, #Documentwomen]"
3,Day 4 of #maxmadness #AirMaxMonth \nNike Air M...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,@MagMr44 @soleguru @nikestore @Nike @SneakerAd...,[]


In [14]:
nike_tw['text'] = nike_tw.text.apply(basic_cleaning)

In [15]:
nike_tw.head()

,text,hashtags
0,nobody cares about nike in russia russia is al...,[]
1,ye green nike hoodie waala two weeks pehle put...,[]
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]"
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd..."
4,thank you sir,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [16]:
nike_tw['brand_attribute'] = nike_tw['text'].apply(data_groups)

In [17]:
nike_tw.head()

,text,hashtags,brand_attribute
0,nobody cares about nike in russia russia is al...,[],quality
1,ye green nike hoodie waala two weeks pehle put...,[],quality
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality
4,thank you sir,[],quality


In [18]:
nike_tw.brand_attribute.value_counts()

quality    23892
price       5384
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [19]:
nike_price = pd.DataFrame(nike_tw.where(nike_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [20]:
nike_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [21]:
nike_price.shape

(5384, 2)

In [22]:
# Exportamos el subset de precio.

nike_price.to_csv('data/nike_price.csv', index = False)

    2.2. Calidad:

In [23]:
nike_quality = pd.DataFrame(nike_tw.where(nike_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [24]:
nike_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [25]:
nike_quality.shape

(23892, 2)

In [26]:
# Exportamos el subset de calidad.

nike_quality.to_csv('data/nike_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Adidas data.

In [27]:
adidas = pd.read_csv('data/adidas.csv')

In [28]:
adidas_tw = pd.DataFrame(adidas.text)

### Exploración inicial de los datos:

In [29]:
adidas_tw.shape

(29816, 1)

In [30]:
adidas_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29816 entries, 0 to 29815
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29816 non-null  object
dtypes: object(1)
memory usage: 233.1+ KB


In [31]:
adidas_tw.describe()

,text
count,29816
unique,29280
top,@whatudohazz nike
freq,60


In [32]:
adidas_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [33]:
adidas_tw.duplicated().sum() # Encontramos 536 valores duplicados.

536

In [34]:
drop_duplicates(adidas_tw) # Eliminamos los valores duplicados.

In [35]:
adidas_tw.shape

(29280, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [36]:
adidas_tw['hashtags'] = adidas_tw.text.apply(extract_hashtags)

In [37]:
adidas_tw.head()

,text,hashtags
0,@Coop8517 Because my lad is a div he won't wea...,[]
1,"@shirtsinthshire beside wrong nameset, its lik...",[]
2,@ChefLerrie @Jakjakph1 Yes dude sa Adidas webs...,[]
3,Footlocker is solely focusing on only selling ...,[]
4,adidas Yeezy Boost 350 Pirate Black V1 (Infant...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [38]:
adidas_tw.head()

,text,hashtags
0,@Coop8517 Because my lad is a div he won't wea...,[]
1,"@shirtsinthshire beside wrong nameset, its lik...",[]
2,@ChefLerrie @Jakjakph1 Yes dude sa Adidas webs...,[]
3,Footlocker is solely focusing on only selling ...,[]
4,adidas Yeezy Boost 350 Pirate Black V1 (Infant...,[]


In [39]:
adidas_tw['text'] = adidas_tw.text.apply(basic_cleaning)

In [40]:
adidas_tw.head()

,text,hashtags
0,because my lad is a div he won t wear adidas o...,[]
1,beside wrong nameset its like fake shirts spac...,[]
2,yes dude sa adidas website mismo yang link sa ...,[]
3,footlocker is solely focusing on only selling ...,[]
4,adidas yeezy boost three five zero pirate blac...,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Ahora llevaremos a cabo este proceso sobre los tweets que mencionan a Adidas.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [41]:
adidas_tw['brand_attribute'] = adidas_tw['text'].apply(data_groups)

In [42]:
adidas_tw.head()

,text,hashtags,brand_attribute
0,because my lad is a div he won t wear adidas o...,[],quality
1,beside wrong nameset its like fake shirts spac...,[],quality
2,yes dude sa adidas website mismo yang link sa ...,[],quality
3,footlocker is solely focusing on only selling ...,[],quality
4,adidas yeezy boost three five zero pirate blac...,[],price


In [43]:
adidas_tw.brand_attribute.value_counts()

quality    23892
price       5388
Name: brand_attribute, dtype: int64

2. Generamos los subsets de Adidas (uno de calidad y otro de precio).

    2.1. Precio:

In [44]:
adidas_price = pd.DataFrame(adidas_tw.where(adidas_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [45]:
adidas_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [46]:
adidas_price.shape

(5388, 2)

In [47]:
# Exportamos el subset de precio.

adidas_price.to_csv('data/adidas_price.csv', index = False)

    2.2. Calidad:

In [48]:
adidas_quality = pd.DataFrame(adidas_tw.where(adidas_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [49]:
adidas_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [50]:
adidas_quality.shape

(23892, 2)

In [51]:
# Exportamos el subset de calidad.

adidas_quality.to_csv('data/adidas_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Asics data.

In [52]:
asics = pd.read_csv('data/asics.csv')

In [53]:
asics_tw = pd.DataFrame(asics.text)

### Exploración inicial de los datos:

In [54]:
asics_tw.shape

(1999, 1)

In [55]:
asics_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1999 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


In [56]:
asics_tw.describe()

,text
count,1999
unique,1976
top,💥INTEL has signed up another #Bitcoin miner fo...
freq,5


In [57]:
asics_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [58]:
asics_tw.duplicated().sum() # Encontramos 23 valores duplicados.

23

In [59]:
drop_duplicates(asics_tw) # Eliminamos los valores duplicados.

In [60]:
asics_tw.shape

(1976, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [61]:
asics_tw['hashtags'] = asics_tw.text.apply(extract_hashtags)

In [62]:
asics_tw.head()

,text,hashtags
0,@LadyLibertas76 Probably around $90 for a pair...,[]
1,"@surferliv Hi, Thanks for your message, we're ...",[]
2,@knivd @aradiogurl The fun about it is that th...,[]
3,Check out this listing I just added to my #Pos...,"[#Poshmark, #shopmycloset]"
4,Check out this listing I just added to my #Pos...,"[#Poshmark, #shopmycloset]"


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [63]:
asics_tw.head()

,text,hashtags
0,@LadyLibertas76 Probably around $90 for a pair...,[]
1,"@surferliv Hi, Thanks for your message, we're ...",[]
2,@knivd @aradiogurl The fun about it is that th...,[]
3,Check out this listing I just added to my #Pos...,"[#Poshmark, #shopmycloset]"
4,Check out this listing I just added to my #Pos...,"[#Poshmark, #shopmycloset]"


In [64]:
asics_tw['text'] = asics_tw.text.apply(basic_cleaning)

In [65]:
asics_tw.head()

,text,hashtags
0,probably around dollars nine zero for a pair o...,[]
1,hi thanks for your message we re really sorry ...,[]
2,the fun about it is that the two eight six mb ...,[]
3,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]"
4,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]"


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [66]:
asics_tw['brand_attribute'] = asics_tw['text'].apply(data_groups)

In [67]:
asics_tw.head()

,text,hashtags,brand_attribute
0,probably around dollars nine zero for a pair o...,[],price
1,hi thanks for your message we re really sorry ...,[],quality
2,the fun about it is that the two eight six mb ...,[],price
3,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality
4,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality


In [68]:
asics_tw.brand_attribute.value_counts()

quality    1540
price       436
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [69]:
asics_price = pd.DataFrame(asics_tw.where(asics_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [70]:
asics_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [71]:
asics_price.shape

(436, 2)

In [72]:
# Exportamos el subset de precio.

asics_price.to_csv('data/asics_price.csv', index = False)

    2.2. Calidad:

In [73]:
asics_quality = pd.DataFrame(asics_tw.where(asics_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [74]:
asics_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [75]:
asics_quality.shape

(1540, 2)

In [76]:
# Exportamos el subset de calidad.

asics_quality.to_csv('data/asics_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Reebok data.

In [77]:
reebok = pd.read_csv('data/reebok.csv')

In [78]:
reebok_tw = pd.DataFrame(reebok.text)

### Exploración inicial de los datos:

In [79]:
reebok_tw.shape

(1996, 1)

In [80]:
reebok_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1996 entries, 0 to 1995
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1996 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


In [81]:
reebok_tw.describe()

,text
count,1996
unique,1953
top,"#NowPlaying Reebok - ""Be More Than Human"" http..."
freq,7


In [82]:
reebok_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [83]:
reebok_tw.duplicated().sum() # Encontramos 43 valores duplicados.

43

In [84]:
drop_duplicates(reebok_tw) # Eliminamos los valores duplicados.

In [85]:
reebok_tw.shape

(1953, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [86]:
reebok_tw['hashtags'] = reebok_tw.text.apply(extract_hashtags)

In [87]:
reebok_tw.head()

,text,hashtags
0,@sportsceneSA @Reebok My wife @prittypriss you...,[]
1,@Lesley_NOPE Sponsored by Reebok!,[]
2,@OneMarineSempFi Thank you. The one in bottom ...,[]
3,https://t.co/mq87iKB9DH\nat 824 save 2474,[]
4,@sportsceneSA @Reebok @EnoughMoela •Thank you ...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [88]:
reebok_tw.head()

,text,hashtags
0,@sportsceneSA @Reebok My wife @prittypriss you...,[]
1,@Lesley_NOPE Sponsored by Reebok!,[]
2,@OneMarineSempFi Thank you. The one in bottom ...,[]
3,https://t.co/mq87iKB9DH\nat 824 save 2474,[]
4,@sportsceneSA @Reebok @EnoughMoela •Thank you ...,[]


In [89]:
reebok_tw['text'] = reebok_tw.text.apply(basic_cleaning)

In [90]:
reebok_tw.head()

,text,hashtags
0,my wife you re strong inspirational caring ama...,[]
1,sponsored by reebok,[]
2,thank you the one in bottom middle pic was ree...,[]
3,at eight two four save two four seven four,[]
4,thank you for always supporting me you was the...,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [91]:
reebok_tw['brand_attribute'] = reebok_tw['text'].apply(data_groups)

In [92]:
reebok_tw.head()

,text,hashtags,brand_attribute
0,my wife you re strong inspirational caring ama...,[],price
1,sponsored by reebok,[],quality
2,thank you the one in bottom middle pic was ree...,[],price
3,at eight two four save two four seven four,[],quality
4,thank you for always supporting me you was the...,[],quality


In [93]:
reebok_tw.brand_attribute.value_counts()

quality    1617
price       336
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [94]:
reebok_price = pd.DataFrame(reebok_tw.where(reebok_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [95]:
reebok_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [96]:
reebok_price.shape

(336, 2)

In [97]:
# Exportamos el subset de precio.

reebok_price.to_csv('data/reebok_price.csv', index = False)

    2.2. Calidad:

In [98]:
reebok_quality = pd.DataFrame(reebok_tw.where(reebok_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [99]:
reebok_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [100]:
reebok_quality.shape

(1617, 2)

In [101]:
# Exportamos el subset de calidad.

reebok_quality.to_csv('data/reebok_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Skechers data.

In [102]:
skechers = pd.read_csv('data/skechers.csv')

In [103]:
skechers_tw = pd.DataFrame(skechers.text)

### Exploración inicial de los datos:

In [104]:
skechers_tw.shape

(1997, 1)

In [105]:
skechers_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1997 entries, 0 to 1996
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1997 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


In [106]:
skechers_tw.describe()

,text
count,1997
unique,1961
top,Check out this listing I just added to my #Pos...
freq,4


In [107]:
skechers_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [108]:
skechers_tw.duplicated().sum() # Encontramos 36 valores duplicados.

36

In [109]:
drop_duplicates(skechers_tw) # Eliminamos los valores duplicados.

In [110]:
skechers_tw.shape

(1961, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [111]:
skechers_tw['hashtags'] = skechers_tw.text.apply(extract_hashtags)

In [112]:
skechers_tw.head()

,text,hashtags
0,I don’t think this is true his mother is heavy...,[]
1,╱ the skechers gif i’m crying 😭😭,[]
2,Skechers USA jumps after President added $3.9M...,[]
3,So good I had to share! Check out all the item...,"[#poshmark, #fashion, #style, #shopmycloset, #..."
4,I was broke had it drippin in Skechers,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [113]:
skechers_tw.head()

,text,hashtags
0,I don’t think this is true his mother is heavy...,[]
1,╱ the skechers gif i’m crying 😭😭,[]
2,Skechers USA jumps after President added $3.9M...,[]
3,So good I had to share! Check out all the item...,"[#poshmark, #fashion, #style, #shopmycloset, #..."
4,I was broke had it drippin in Skechers,[]


In [114]:
skechers_tw['text'] = skechers_tw.text.apply(basic_cleaning)

In [115]:
skechers_tw.head()

,text,hashtags
0,i don t think this is true his mother is heavy...,[]
1,the skechers gif i m crying,[]
2,skechers usa jumps after president added dolla...,[]
3,so good i had to share check out all the items...,"[#poshmark, #fashion, #style, #shopmycloset, #..."
4,i was broke had it drippin in skechers,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [116]:
skechers_tw['brand_attribute'] = skechers_tw['text'].apply(data_groups)

In [117]:
skechers_tw.head()

,text,hashtags,brand_attribute
0,i don t think this is true his mother is heavy...,[],quality
1,the skechers gif i m crying,[],quality
2,skechers usa jumps after president added dolla...,[],price
3,so good i had to share check out all the items...,"[#poshmark, #fashion, #style, #shopmycloset, #...",quality
4,i was broke had it drippin in skechers,[],quality


In [118]:
skechers_tw.brand_attribute.value_counts()

quality    1654
price       307
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [119]:
skechers_price = pd.DataFrame(skechers_tw.where(skechers_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [120]:
skechers_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [121]:
skechers_price.shape

(307, 2)

In [122]:
# Exportamos el subset de precio.

skechers_price.to_csv('data/skechers_price.csv', index = False)

    2.2. Calidad:

In [123]:
skechers_quality = pd.DataFrame(skechers_tw.where(skechers_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [124]:
skechers_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [125]:
skechers_quality.shape

(1654, 2)

In [126]:
# Exportamos el subset de calidad.

skechers_quality.to_csv('data/skechers_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Under Armour data.

In [127]:
under_armour = pd.read_csv('data/under_armour.csv')

In [128]:
under_armour_tw = pd.DataFrame(under_armour.text)

### Exploración inicial de los datos:

In [129]:
under_armour_tw.shape

(1998, 1)

In [130]:
under_armour_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1998 entries, 0 to 1997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1998 non-null   object
dtypes: object(1)
memory usage: 15.7+ KB


In [131]:
under_armour_tw.describe()

,text
count,1998
unique,1940
top,"Too little, too late, you'll never change! Som..."
freq,5


In [132]:
under_armour_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [133]:
under_armour_tw.duplicated().sum() # Encontramos 58 valores duplicados.

58

In [134]:
drop_duplicates(under_armour_tw) # Eliminamos los valores duplicados.

In [135]:
under_armour_tw.shape

(1940, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [136]:
under_armour_tw['hashtags'] = under_armour_tw.text.apply(extract_hashtags)

In [137]:
under_armour_tw.head()

,text,hashtags
0,“ I would…appreciate the help with the armour....,[]
1,@EuphoriantForce Boots (Under Armour Stellar)\...,[]
2,Under Armour NEW Blue Womens Size XL Compressi...,"[#eBay🇺🇸, #Women, #Pants]"
3,Boycott Under Armour for Promoting Trophy-Hunt...,[]
4,Under Armour Men's Rival Fleece Big Logo Hoodi...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [138]:
under_armour_tw.head()

,text,hashtags
0,“ I would…appreciate the help with the armour....,[]
1,@EuphoriantForce Boots (Under Armour Stellar)\...,[]
2,Under Armour NEW Blue Womens Size XL Compressi...,"[#eBay🇺🇸, #Women, #Pants]"
3,Boycott Under Armour for Promoting Trophy-Hunt...,[]
4,Under Armour Men's Rival Fleece Big Logo Hoodi...,[]


In [139]:
under_armour_tw['text'] = under_armour_tw.text.apply(basic_cleaning)

In [140]:
under_armour_tw.head()

,text,hashtags
0,i would appreciate the help with the armour he...,[]
1,boots under armour stellar then usually some s...,[]
2,under armour new blue womens size xl compressi...,"[#eBay🇺🇸, #Women, #Pants]"
3,boycott under armour for promoting trophy hunt...,[]
4,under armour men s rival fleece big logo hoodie,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [141]:
under_armour_tw['brand_attribute'] = under_armour_tw['text'].apply(data_groups)

In [142]:
under_armour_tw.head()

,text,hashtags,brand_attribute
0,i would appreciate the help with the armour he...,[],quality
1,boots under armour stellar then usually some s...,[],quality
2,under armour new blue womens size xl compressi...,"[#eBay🇺🇸, #Women, #Pants]",price
3,boycott under armour for promoting trophy hunt...,[],quality
4,under armour men s rival fleece big logo hoodie,[],quality


In [143]:
under_armour_tw.brand_attribute.value_counts()

quality    1594
price       346
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [144]:
under_armour_price = pd.DataFrame(under_armour_tw.where(under_armour_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [145]:
under_armour_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [146]:
under_armour_price.shape

(346, 2)

In [147]:
# Exportamos el subset de precio.

under_armour_price.to_csv('data/under_armour_price.csv', index = False)

    2.2. Calidad:

In [148]:
under_armour_quality = pd.DataFrame(under_armour_tw.where(under_armour_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [149]:
under_armour_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [150]:
under_armour_quality.shape

(1594, 2)

In [151]:
# Exportamos el subset de calidad.

under_armour_quality.to_csv('data/under_armour_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Umbro data.

In [152]:
umbro = pd.read_csv('data/umbro.csv')

In [153]:
umbro_tw = pd.DataFrame(umbro.text)

### Exploración inicial de los datos:

In [154]:
umbro_tw.shape

(698, 1)

In [155]:
umbro_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 698 entries, 0 to 697
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    698 non-null    object
dtypes: object(1)
memory usage: 5.6+ KB


In [156]:
umbro_tw.describe()

,text
count,698
unique,698
top,@JeffreyLloydXXX @Orsini_Umbro Let s do flip f...
freq,1


In [157]:
umbro_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [158]:
umbro_tw.duplicated().sum() # No existen valores duplicados.

0

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [159]:
umbro_tw['hashtags'] = umbro_tw.text.apply(extract_hashtags)

In [160]:
umbro_tw.head()

,text,hashtags
0,@JeffreyLloydXXX @Orsini_Umbro Let s do flip f...,[]
1,@sportbible Umbro ball 🤔,[]
2,Manchester United Shirt 1992-1994 Umbro Small ...,[]
3,Celtic football shirt\n1999 away season \nLarg...,[]
4,@sid_lambert an umbro football..,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [161]:
umbro_tw.head()

,text,hashtags
0,@JeffreyLloydXXX @Orsini_Umbro Let s do flip f...,[]
1,@sportbible Umbro ball 🤔,[]
2,Manchester United Shirt 1992-1994 Umbro Small ...,[]
3,Celtic football shirt\n1999 away season \nLarg...,[]
4,@sid_lambert an umbro football..,[]


In [162]:
umbro_tw['text'] = umbro_tw.text.apply(basic_cleaning)

In [163]:
umbro_tw.head()

,text,hashtags
0,let s do flip flop,[]
1,umbro ball,[]
2,manchester united shirt one nine nine two one ...,[]
3,celtic football shirt one nine nine nine away ...,[]
4,an umbro football,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [164]:
umbro_tw['brand_attribute'] = umbro_tw['text'].apply(data_groups)

In [165]:
umbro_tw.head()

,text,hashtags,brand_attribute
0,let s do flip flop,[],quality
1,umbro ball,[],quality
2,manchester united shirt one nine nine two one ...,[],quality
3,celtic football shirt one nine nine nine away ...,[],price
4,an umbro football,[],quality


In [166]:
umbro_tw.brand_attribute.value_counts()

quality    621
price       77
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [167]:
umbro_price = pd.DataFrame(umbro_tw.where(umbro_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [168]:
umbro_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [169]:
umbro_price.shape

(77, 2)

In [170]:
# Exportamos el subset de precio.

umbro_price.to_csv('data/umbro_price.csv', index = False)

    2.2. Calidad:

In [171]:
umbro_quality = pd.DataFrame(umbro_tw.where(umbro_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [172]:
umbro_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [173]:
umbro_quality.shape

(621, 2)

In [174]:
# Exportamos el subset de calidad.

umbro_quality.to_csv('data/umbro_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Puma data.

In [175]:
puma = pd.read_csv('data/puma.csv')

In [176]:
puma_tw = pd.DataFrame(puma.text)

### Exploración inicial de los datos:

In [177]:
puma_tw.shape

(5090, 1)

In [178]:
puma_tw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5090 entries, 0 to 5089
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5090 non-null   object
dtypes: object(1)
memory usage: 39.9+ KB


In [179]:
puma_tw.describe()

,text
count,5090
unique,4984
top,Puma\n\nI vote for #Butter for #BestMusicVideo...
freq,10


In [180]:
puma_tw.isnull().sum()  # No existen nulos.

text    0
dtype: int64

In [181]:
puma_tw.duplicated().sum() # Encontramos 106 valores duplicados.

106

In [182]:
drop_duplicates(puma_tw) # Eliminamos los valores duplicados.

In [183]:
puma_tw.shape

(4984, 1)

### Hashtags.

Queremos extraer los hastags de los tweets y almacenarlos en una nueva columna del DataFrame. Esto nos permitirá llevar a cabo un análisis específico de los hashtags una vez hecho el análisis del sentimiento de los tweets, relacionando el sentimiento con los hashtags utilizados en el tweet.

In [184]:
puma_tw['hashtags'] = puma_tw.text.apply(extract_hashtags)

In [185]:
puma_tw.head()

,text,hashtags
0,#Puma is coming to Roblox. https://t.co/h2x1Wk...,[#Puma]
1,[RANKING BEST TXT CHOREOGRAPHIES] \n\n7. Puma ...,[]
2,@PUMA @CatBloxStudios LFG PUMA,[]
3,"Palestine is a feminist issue. @HP, @HPE, @AXA...","[#BDS, #InternationalWomensDay, #IWD2022]"
4,@kappie777 @PUMA @PUMASouthAfrica Happy women'...,[]


### Preparing Data.

**Limpieza básica:**
- Pasamos todo a minúsculas.
- Eliminamos usuarios ('@ + user' ya que en principio el nombre de los usuarios no aporta información relevante para el modelo).
- Eliminamos urls.
- Convertimos dígitos a números escritos.
- Convertimos $ en 'dollars' y € en 'euros.
- Eliminamos valores no alfabéticos: # (aunque manteniendo el texto que viene después ya que puede contener información relevante), signos de puntuación...

Esta limpieza se ejecutará a través de nuestra función `basic_cleaning()`.

In [186]:
puma_tw.head()

,text,hashtags
0,#Puma is coming to Roblox. https://t.co/h2x1Wk...,[#Puma]
1,[RANKING BEST TXT CHOREOGRAPHIES] \n\n7. Puma ...,[]
2,@PUMA @CatBloxStudios LFG PUMA,[]
3,"Palestine is a feminist issue. @HP, @HPE, @AXA...","[#BDS, #InternationalWomensDay, #IWD2022]"
4,@kappie777 @PUMA @PUMASouthAfrica Happy women'...,[]


In [187]:
puma_tw['text'] = puma_tw.text.apply(basic_cleaning)

In [188]:
puma_tw.head()

,text,hashtags
0,puma is coming to roblox,[#Puma]
1,ranking best txt choreographies seven puma six...,[]
2,lfg puma,[]
3,palestine is a feminist issue more are complic...,"[#BDS, #InternationalWomensDay, #IWD2022]"
4,happy women s day keep shining mam could i get...,[]


### Subsets.

En esta primera versión de la herramienta construiremos el mapa de posicionamiento en base a dos atributos de marca (variables): calidad y precio. Estos son los dos atributos básicos más utilizados de manera general en los mapas de posicionamiento (los mapas más sofisticados introducen ya otros atributos más específicos).

Para poder completar el proceso de mapeo, lo primero que debemos hacer es dividir los datos (tweets recopilados) en dos subsets: los que hacen referencia al precio y los que hacen referencia a la calidad. Una vez subdivididos, ya podremos introducir los datos en el modelo NLP para evaluar el sentimiento en cada subset.

La subdivisión nos permite diferenciar el sentimiento de los tweets que hablan del precio, del sentimiento en torno a la calidad general de la marca en cuestión. Esto es lo que haremos en esta etapa del proceso, en este caso sobre los tweets que mencionan a Nike.

1. Generamos una nueva columna que recoge la temática del tweet (atributo al que hace refencia).

In [189]:
puma_tw['brand_attribute'] = puma_tw['text'].apply(data_groups)

In [190]:
puma_tw.head()

,text,hashtags,brand_attribute
0,puma is coming to roblox,[#Puma],price
1,ranking best txt choreographies seven puma six...,[],quality
2,lfg puma,[],quality
3,palestine is a feminist issue more are complic...,"[#BDS, #InternationalWomensDay, #IWD2022]",quality
4,happy women s day keep shining mam could i get...,[],quality


In [191]:
puma_tw.brand_attribute.value_counts()

quality    4132
price       852
Name: brand_attribute, dtype: int64

2. Exportamos los subsets de Nike (uno de calidad y otro de precio).

    2.1. Precio:

In [192]:
puma_price = pd.DataFrame(puma_tw.where(puma_tw.brand_attribute=='price').dropna().drop('brand_attribute',axis=1))

In [193]:
puma_price.isnull().sum()

text        0
hashtags    0
dtype: int64

In [194]:
puma_price.shape

(852, 2)

In [195]:
# Exportamos el subset de precio.

puma_price.to_csv('data/puma_price.csv', index = False)

    2.2. Calidad:

In [196]:
puma_quality = pd.DataFrame(puma_tw.where(puma_tw.brand_attribute=='quality').dropna().drop('brand_attribute',axis=1))

In [197]:
puma_quality.isnull().sum()

text        0
hashtags    0
dtype: int64

In [198]:
puma_quality.shape

(4132, 2)

In [199]:
# Exportamos el subset de calidad.

puma_quality.to_csv('data/puma_quality.csv', index = False)

**Nota importante:**

Aunque hemos exportado cada subset en archivos csv independientes (lo tenemos guardado de manera desagregada), en realidad trabajaremos con un único DataFrame que contenga todos los tweets analizados (los de todas las marcas y todos los atributos). En este DataFrame recogeremos en una columna la marca a la que hacen referencia y en otra columna el atributo al que se refieren (calidad y precio). Esto lo hacemos así porque nos permitirá agilizar mucho los procesos de visualización de resultados y obtener gráficos de mejor calidad (podremos filtrar en función de la marca y el atributo, al tiempo que manejamos un único DataFrame para mayor eficiencia).

## Dataset Completo.

Primero generamos una nueva columna en la que se recoja la marca a la que hace referencia el tweet (todos los valores de la columna serán 'nike' para el DataFrame de Nike, 'adidas' para el DataFrame de Adidas...).

#### Columna 'brand' para los datos de Nike.

In [200]:
nike_tw.head()

,text,hashtags,brand_attribute
0,nobody cares about nike in russia russia is al...,[],quality
1,ye green nike hoodie waala two weeks pehle put...,[],quality
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality
4,thank you sir,[],quality


In [201]:
nike_tw = nike_tw.assign(brand='nike')

In [202]:
nike_tw.head()

,text,hashtags,brand_attribute,brand
0,nobody cares about nike in russia russia is al...,[],quality,nike
1,ye green nike hoodie waala two weeks pehle put...,[],quality,nike
2,nike okundaye also knowns nike twins seven sev...,"[#womengiant, #Documentwomen]",quality,nike
3,day four of maxmadness air max month nike air ...,"[#maxmadness, #AirMaxMonth, #airmaxgang, #kotd...",quality,nike
4,thank you sir,[],quality,nike


#### Columna 'brand' para los datos de Adidas.

In [203]:
adidas_tw.head()

,text,hashtags,brand_attribute
0,because my lad is a div he won t wear adidas o...,[],quality
1,beside wrong nameset its like fake shirts spac...,[],quality
2,yes dude sa adidas website mismo yang link sa ...,[],quality
3,footlocker is solely focusing on only selling ...,[],quality
4,adidas yeezy boost three five zero pirate blac...,[],price


In [204]:
adidas_tw = adidas_tw.assign(brand='adidas')

In [205]:
adidas_tw.head()

,text,hashtags,brand_attribute,brand
0,because my lad is a div he won t wear adidas o...,[],quality,adidas
1,beside wrong nameset its like fake shirts spac...,[],quality,adidas
2,yes dude sa adidas website mismo yang link sa ...,[],quality,adidas
3,footlocker is solely focusing on only selling ...,[],quality,adidas
4,adidas yeezy boost three five zero pirate blac...,[],price,adidas


#### Columna 'brand' para los datos de Asics.

In [206]:
asics_tw.head()

,text,hashtags,brand_attribute
0,probably around dollars nine zero for a pair o...,[],price
1,hi thanks for your message we re really sorry ...,[],quality
2,the fun about it is that the two eight six mb ...,[],price
3,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality
4,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality


In [207]:
asics_tw = asics_tw.assign(brand='asics')

In [208]:
asics_tw.head()

,text,hashtags,brand_attribute,brand
0,probably around dollars nine zero for a pair o...,[],price,asics
1,hi thanks for your message we re really sorry ...,[],quality,asics
2,the fun about it is that the two eight six mb ...,[],price,asics
3,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality,asics
4,check out this listing i just added to my posh...,"[#Poshmark, #shopmycloset]",quality,asics


#### Columna 'brand' para los datos de Reebok.

In [209]:
reebok_tw.head()

,text,hashtags,brand_attribute
0,my wife you re strong inspirational caring ama...,[],price
1,sponsored by reebok,[],quality
2,thank you the one in bottom middle pic was ree...,[],price
3,at eight two four save two four seven four,[],quality
4,thank you for always supporting me you was the...,[],quality


In [210]:
reebok_tw = reebok_tw.assign(brand='reebok')

In [211]:
reebok_tw.head()

,text,hashtags,brand_attribute,brand
0,my wife you re strong inspirational caring ama...,[],price,reebok
1,sponsored by reebok,[],quality,reebok
2,thank you the one in bottom middle pic was ree...,[],price,reebok
3,at eight two four save two four seven four,[],quality,reebok
4,thank you for always supporting me you was the...,[],quality,reebok


#### Columna 'brand' para los datos de Skechers.

In [212]:
skechers_tw.head()

,text,hashtags,brand_attribute
0,i don t think this is true his mother is heavy...,[],quality
1,the skechers gif i m crying,[],quality
2,skechers usa jumps after president added dolla...,[],price
3,so good i had to share check out all the items...,"[#poshmark, #fashion, #style, #shopmycloset, #...",quality
4,i was broke had it drippin in skechers,[],quality


In [213]:
skechers_tw = skechers_tw.assign(brand='skechers')

In [214]:
skechers_tw.head()

,text,hashtags,brand_attribute,brand
0,i don t think this is true his mother is heavy...,[],quality,skechers
1,the skechers gif i m crying,[],quality,skechers
2,skechers usa jumps after president added dolla...,[],price,skechers
3,so good i had to share check out all the items...,"[#poshmark, #fashion, #style, #shopmycloset, #...",quality,skechers
4,i was broke had it drippin in skechers,[],quality,skechers


#### Columna 'brand' para los datos de Under Armour.

In [215]:
under_armour_tw.head()

,text,hashtags,brand_attribute
0,i would appreciate the help with the armour he...,[],quality
1,boots under armour stellar then usually some s...,[],quality
2,under armour new blue womens size xl compressi...,"[#eBay🇺🇸, #Women, #Pants]",price
3,boycott under armour for promoting trophy hunt...,[],quality
4,under armour men s rival fleece big logo hoodie,[],quality


In [216]:
under_armour_tw = under_armour_tw.assign(brand='under armour')

In [217]:
under_armour_tw.head()

,text,hashtags,brand_attribute,brand
0,i would appreciate the help with the armour he...,[],quality,under armour
1,boots under armour stellar then usually some s...,[],quality,under armour
2,under armour new blue womens size xl compressi...,"[#eBay🇺🇸, #Women, #Pants]",price,under armour
3,boycott under armour for promoting trophy hunt...,[],quality,under armour
4,under armour men s rival fleece big logo hoodie,[],quality,under armour


#### Columna 'brand' para los datos de Umbro.

In [218]:
umbro_tw.head()

,text,hashtags,brand_attribute
0,let s do flip flop,[],quality
1,umbro ball,[],quality
2,manchester united shirt one nine nine two one ...,[],quality
3,celtic football shirt one nine nine nine away ...,[],price
4,an umbro football,[],quality


In [219]:
umbro_tw = umbro_tw.assign(brand='umbro')

In [220]:
umbro_tw.head()

,text,hashtags,brand_attribute,brand
0,let s do flip flop,[],quality,umbro
1,umbro ball,[],quality,umbro
2,manchester united shirt one nine nine two one ...,[],quality,umbro
3,celtic football shirt one nine nine nine away ...,[],price,umbro
4,an umbro football,[],quality,umbro


#### Columna 'brand' para los datos de Puma.

In [221]:
puma_tw.head()

,text,hashtags,brand_attribute
0,puma is coming to roblox,[#Puma],price
1,ranking best txt choreographies seven puma six...,[],quality
2,lfg puma,[],quality
3,palestine is a feminist issue more are complic...,"[#BDS, #InternationalWomensDay, #IWD2022]",quality
4,happy women s day keep shining mam could i get...,[],quality


In [222]:
puma_tw = puma_tw.assign(brand='puma')

In [223]:
puma_tw.head()

,text,hashtags,brand_attribute,brand
0,puma is coming to roblox,[#Puma],price,puma
1,ranking best txt choreographies seven puma six...,[],quality,puma
2,lfg puma,[],quality,puma
3,palestine is a feminist issue more are complic...,"[#BDS, #InternationalWomensDay, #IWD2022]",quality,puma
4,happy women s day keep shining mam could i get...,[],quality,puma


### Generamos dataset completo.

In [224]:
brands_tw = pd.concat([nike_tw, adidas_tw, asics_tw, reebok_tw, skechers_tw, under_armour_tw, umbro_tw, puma_tw])

In [225]:
brands_tw.brand.value_counts()

adidas          29280
nike            29276
puma             4984
asics            1976
skechers         1961
reebok           1953
under armour     1940
umbro             698
Name: brand, dtype: int64

### Exportamos el dataset completo.

In [226]:
brands_tw.to_csv('data/sports_equipment_brands.csv', index = False)

In [227]:
# Exportamos con pickle para mantener todas las propiedades del DataFrame.

brands_tw.to_pickle('data/sports_equipment_brands.pkl')